In [1]:
from Synthesizer import synthesize

In [2]:
programs = synthesize(
    input_output_simple_list=[
        ([1, 5, 2], [1, 5]),
        ([3, 9], [3])
    ],
    variable_names=['x'],
    search_space_file='utils/grammars/ListGrammar.txt',
    max_height=2,
    statistics=True
)

The synthesizer searched 52 programs up to height #1.


In [3]:
programs

['x[0:-1:None]']